In [1]:
#%%
try:
    import os
    import sys
    import warnings
    warnings.filterwarnings("ignore")
    print(f'Default working directory {os.getcwd()}')
    #change the directory to working directory
    #path =r"C:\Work_Anaconda\TripAdvisor\All_script_voting"
    path =r"C:\Users\arshraj.randhawa\Desktop\work2022\reikit model building"
    os.chdir(path)
    print(f'Current working directory {os.getcwd()}')
    #check the files in directory
    # print(f'Files in the current working directory are:\n{os.listdir()}')
except Exception as e:
    print(f'Error while setting up working directory : {sys.exc_info()[0]} , {e}')

Default working directory C:\Users\arshraj.randhawa\Desktop\work2022\reikit model building
Current working directory C:\Users\arshraj.randhawa\Desktop\work2022\reikit model building


In [2]:

print('Importing Libraries & functions')
try :
    import warnings
    warnings.filterwarnings("ignore")
    import spacy
    #if spacy is not installed >>
    # pip install -U spacy
    # python -m spacy download en_core_web_lg
    import pandas as pd
    import joblib
    # pip install joblib
    import numpy as np
    import glob
    from sklearn.metrics import accuracy_score
    from support_file2 import *
    from sklearn.preprocessing import LabelEncoder
    from nltk.tokenize import sent_tokenize
    from sklearn.model_selection import train_test_split
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn import metrics
    from sklearn.metrics import confusion_matrix
    from sklearn.feature_selection import SelectFromModel
    from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier
    from sklearn.linear_model import SGDClassifier
    from sklearn.naive_bayes import ComplementNB
    from keras.utils import to_categorical
    from keras.models import Sequential
    from keras.layers import Dense, Dropout
    from keras.callbacks import EarlyStopping
    print('Librabries and functions imported successfull')
except Exception as e:
    print(f'Error while importing librabries {sys.exc_info()[0]} , {e}')

Importing Libraries & functions


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\arshraj.randhawa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Using TensorFlow backend.


Librabries and functions imported successfull


In [3]:
print('Importing new training data from Database')
try:
    #read data from the source
    # df = pd.read_sql_query()
    df = pd.read_excel('training_data_airwick.xlsx')

#         df = pd.read_pickle('ta_snippet_10032021.pkl')
    #only the 4 columns are necessary
    df = df[['ReviewID','Month', 'Title', 'Review', 'FullText','Rating','Title_Review','Level1','Level2','Level3']]
    head, shape, dtyp, colms, nul = dfinfo(df)
    #sort
    df.columns = [col.lower() for col in colms]
    df.sort_values(['reviewid', 'title'], inplace=True)
    df.dropna(subset=['review'],inplace=True)
    df.reset_index(drop=True, inplace=True)

    #lowering all text columns
    for col in ['title','month','review', 'fulltext', 'title_review']:
        df[col] = df[col].str.lower()
        df[col] = df[col].str.strip()
    del col
    df=df[df.month=="august"]
    df.reset_index(drop=True,inplace=True)
except Exception as e:
    print(f"Error while fetching the data data : {sys.exc_info()[0]} : {e}")
# df = df[['userreviewid', 'reviewtext']].drop_duplicates('userreviewid')

Importing new training data from Database


In [4]:
df

,reviewid,month,title,review,fulltext,rating,title_review,level1,level2,level3
0,214636497,august,yea but eh,i like the diffusers and some scents small gre...,NaN,3,yea but eh[$]i like the diffusers and some sce...,Pricing,Expensive,Expensive
1,215105472,august,won't work as described,i bought this product and inserted everything ...,NaN,1,won't work as described[$]i bought this produc...,Manufacturing & Design,Poor Quality/Defective Product/Device Issue,Stopped/Not working
2,215280894,august,stops working,i purchased 2 of these and they work for a lit...,NaN,1,stops working[$]i purchased 2 of these and the...,Manufacturing & Design,Poor Quality/Defective Product/Device Issue,Stopped/Not working
3,215343666,august,great product when it works,seems like i am not the only one with this iss...,NaN,2,great product when it works[$]seems like i am ...,Manufacturing & Design,Poor Quality/Defective Product/Device Issue,Stopped/Not working
4,215384812,august,air freshener,smells great and lasts longer than expected. g...,NaN,5,air freshener[$]smells great and lasts longer ...,Functionality,Pleasant Fragrance,Pleasant Fragrance
...,...,...,...,...,...,...,...,...,...,...
2384,RZS8T6TQXZPX7,august,good scent,loved the scent,NaN,5,good scent[$]loved the scent,Functionality,Pleasant Fragrance,Pleasant Fragrance
2385,RZTWLOOZF0L3D,august,love this scent!,this has to be one of my very favorite scents ...,NaN,5,love this scent![$]this has to be one of my ve...,Functionality,Pleasant Fragrance,Pleasant Fragrance
2386,RZTWUIZBZN1KV,august,good product,good product,NaN,5,good product[$]good product,Brand Perception,Positive Brand Perception,Positive Brand Perception
2387,RZW99KP3T0KQL,august,no scent,i usually buy my air wick refills in bulk from...,NaN,1,no scent[$]i usually buy my air wick refills i...,Functionality,Effectiveness in Getting the Desired Output,Weak/No Smell


In [5]:

# =========================Pre processing======================================

print('Preproceing starts...')
try:
    df['review'] = df['review'].apply(cleantext)
    df = preprocess(df, 'review')
    df['cc1'] = df['cc1'].replace('', np.nan)
    head, shape, dtyp, colms, nul = dfinfo(df)
    nullreco = df[df['cc1'].isna()]
    #drop nul values
    dupli = df[df.duplicated(subset = ['reviewid', 'cc1'], keep=False)]
    df = df.drop_duplicates(subset=['reviewid', 'cc1']).reset_index(drop=True)
    df.reset_index(drop=True, inplace=True)
    #preparing dictionary d
    head, shape, dtyp, colms, nul = dfinfo(df)
    print("dropping null rows")
    df.dropna(subset=['cc1'],inplace=True)
    colms.remove('cc')
    d = uniq(df[colms])
    colms = list(df.columns)
    df['word'] = df['cc1'].str.split().str.len()
    df['len'] = df['cc1'].apply(len)
#     df = df[['reviewid', 'snippet', 'theme', 'sentiment', 'cc', 'cc1', '#word', 'len']]
    print('\nPreproceing of new Done')
except Exception as e:
    print(f'Error in preprocessing : {sys.exc_info()[0]} , {e}')


Preproceing starts...
removed emojis
removed punctuation
removed blank lines
lematization of text is in process...
lemmmatized text
removed stop words
removed single words
removed digits
removed white spaces
dropping null rows
reviewid>month>title>review>fulltext>rating>title_review>level1>level2>level3>cc1>
Preproceing of new Done


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2376 entries, 0 to 2388
Data columns (total 14 columns):
reviewid        2376 non-null object
month           2376 non-null object
title           2370 non-null object
review          2376 non-null object
fulltext        0 non-null object
rating          2376 non-null int64
title_review    2376 non-null object
level1          2376 non-null object
level2          2376 non-null object
level3          2376 non-null object
cc              2376 non-null object
cc1             2376 non-null object
word            2376 non-null int64
len             2376 non-null int64
dtypes: int64(3), object(11)
memory usage: 278.4+ KB


In [7]:

# =============================================================================
try:
    print('Loading ML models')
    x = df['cc1']
    tf = joblib.load('tf')
    x_tf  = tf.transform(x)
    
    selector = joblib.load('selector')
    x_tf = selector.transform(x_tf)

    
except Exception as e:
    print(f"Error while prediction from ML models' : {sys.exc_info()[0]} : {e}")


Loading ML models


In [8]:
try:
    print('Running individual model with probability')
    # ======================================    
    m_sgd = joblib.load('m_sgd')
    m_gbt = joblib.load('m_gbt')
    m_cnb = joblib.load('m_cnb')
    le = joblib.load('le')
    # ======================================
    
    # ======================================
    print('Running the prediction on data')    
    # psgd = m_sgd.predict(x_tf)   
    # pgbt = m_gbt.predict(x_tf)
    # pcnb = m_cnb.predict(x_tf)
    # ======================================

    # ======================================
    pred_sgd = m_sgd.predict_proba(x_tf)
    pred = pd.DataFrame(data=pred_sgd, columns=le.classes_)
    pred['max'] = np.max(pred, axis=1)
    pred['them'] = pred.idxmax(axis=1)
    df['maxsgd'] = pred['max'].tolist()
    df['psgd'] = pred['them'].tolist()
    # ======================================    

    # ======================================
    pred_gbt = m_gbt.predict_proba(x_tf)
    pred = pd.DataFrame(data=pred, columns=le.classes_)
    pred['max'] = np.max(pred, axis=1)
    pred['them'] = pred.idxmax(axis=1)
    df['maxgbt'] = pred['max'].tolist()
    df['pgbt'] = pred['them'].tolist()
    # ======================================
    
    # ======================================
    pred_nb = m_cnb.predict_proba(x_tf)
    pred = pd.DataFrame(data=pred_nb, columns=le.classes_)
    pred['max'] = np.max(pred, axis=1)
    pred['them'] = pred.idxmax(axis=1)
    df['maxcnb'] = pred['max'].tolist()
    df['pcnb'] = pred['them'].tolist()
#     ======================================
    
    # psgd = le.inverse_transform(psgd)
    # pgbt = le.inverse_transform(pgbt)
    # pcnb = le.inverse_transform(pcnb)
    
    # df1['psgd'] = psgd.tolist()
    # df1['pgbt'] = pgbt.tolist()
    # df1['pcnb'] = pcnb.tolist()
    
    print('Successfully predicted results of 3 models')
except Exception as e:
    print(f"Error while prediction from ML models' : {sys.exc_info()[0]} : {e}")


Running individual model with probability
Running the prediction on data
Successfully predicted results of 3 models


In [9]:
df

,reviewid,month,title,review,fulltext,rating,title_review,level1,level2,level3,cc,cc1,word,len,maxsgd,psgd,maxgbt,pgbt,maxcnb,pcnb
0,214636497,august,yea but eh,i like the diffusers and some scents small gre...,NaN,3,yea but eh[$]i like the diffusers and some sce...,Pricing,Expensive,Expensive,"[like, diffuser, scent, small, great, limit, v...",like diffuser scent small great limit variety ...,17,105,0.382206,Product Features,0.382206,Product Features,0.165960,Functionality
1,215105472,august,won't work as described,i bought this product and inserted everything ...,NaN,1,won't work as described[$]i bought this produc...,Manufacturing & Design,Poor Quality/Defective Product/Device Issue,Stopped/Not working,"[buy, product, insert, correctly, , slap, jig...",buy product insert correctly slap jiggle lever...,13,89,0.343989,Brand Perception,0.343989,Brand Perception,0.151819,Manufacturing & Design
2,215280894,august,stops working,i purchased 2 of these and they work for a lit...,NaN,1,stops working[$]i purchased 2 of these and the...,Manufacturing & Design,Poor Quality/Defective Product/Device Issue,Stopped/Not working,"[purchase, 2, work, little, stop, , love, sc...",purchase work little stop love scent disappoin...,13,84,0.495424,Manufacturing & Design,0.495424,Manufacturing & Design,0.169097,Functionality
3,215343666,august,great product when it works,seems like i am not the only one with this iss...,NaN,2,great product when it works[$]seems like i am ...,Manufacturing & Design,Poor Quality/Defective Product/Device Issue,Stopped/Not working,"[like, not, issue, , ’, work, not, , stop, ...",like not issue work not stop work open play wo...,12,66,0.624465,Manufacturing & Design,0.624465,Manufacturing & Design,0.253057,Manufacturing & Design
4,215384812,august,air freshener,smells great and lasts longer than expected. g...,NaN,5,air freshener[$]smells great and lasts longer ...,Functionality,Pleasant Fragrance,Pleasant Fragrance,"[smell, great, longer, expect, , great, buy]",smell great longer expect great buy,6,35,0.485312,Product Features,0.485312,Product Features,0.177441,Functionality
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2384,RZS8T6TQXZPX7,august,good scent,loved the scent,NaN,5,good scent[$]loved the scent,Functionality,Pleasant Fragrance,Pleasant Fragrance,"[love, scent]",love scent,2,10,1.000000,Functionality,1.000000,Functionality,0.425757,Functionality
2385,RZTWLOOZF0L3D,august,love this scent!,this has to be one of my very favorite scents ...,NaN,5,love this scent![$]this has to be one of my ve...,Functionality,Pleasant Fragrance,Pleasant Fragrance,"[favorite, scent]",favorite scent,2,14,1.000000,Functionality,1.000000,Functionality,0.390019,Functionality
2386,RZTWUIZBZN1KV,august,good product,good product,NaN,5,good product[$]good product,Brand Perception,Positive Brand Perception,Positive Brand Perception,"[good, product]",good product,2,12,0.820293,Brand Perception,0.820293,Brand Perception,0.251378,Brand Perception
2387,RZW99KP3T0KQL,august,no scent,i usually buy my air wick refills in bulk from...,NaN,1,no scent[$]i usually buy my air wick refills i...,Functionality,Effectiveness in Getting the Desired Output,Weak/No Smell,"[usually, buy, air, wick, refill, bulk, local,...",usually buy air wick refill bulk local store d...,22,137,0.462239,Functionality,0.462239,Functionality,0.184264,Functionality


In [10]:
df.to_csv('Final_output.csv', index=False)
print("results dumped to csv")

results dumped to csv
